In [1]:
import streamlit as st
import pandas as pd
import numpy as np

import gensim.downloader as api

In [2]:
st.title('Word Embeddings')

2022-04-01 16:27:57.923 
  command:

    streamlit run /Users/kate/opt/anaconda3/envs/metis/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [4]:
model_wikipedia50 = api.load("glove-wiki-gigaword-50")

2022-04-07 16:20:39.588 INFO    gensim.models.keyedvectors: loading projection weights from /Users/kate/gensim-data/glove-wiki-gigaword-50/glove-wiki-gigaword-50.gz
2022-04-07 16:20:49.916 INFO    gensim.utils: KeyedVectors lifecycle event {'msg': 'loaded (400000, 50) matrix of type float32 from /Users/kate/gensim-data/glove-wiki-gigaword-50/glove-wiki-gigaword-50.gz', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-04-07T16:20:49.916777', 'gensim': '4.0.1', 'python': '3.9.9 | packaged by conda-forge | (main, Dec 20 2021, 02:41:37) \n[Clang 11.1.0 ]', 'platform': 'macOS-12.3-x86_64-i386-64bit', 'event': 'load_word2vec_format'}


In [4]:
# Create a text element and let the reader know the data is loading.
data_load_state = st.text('Loading data...')
# Load pre-trained embeddings
data = model_wikipedia50
# Notify the reader that the data was successfully loaded.
data_load_state.text('Loading data...done!')

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [5]:
#create interactive 3d graph of word embeddings

def append_list(sim_words, words):
    
    list_of_words = []
    
    for i in range(len(sim_words)):
        
        sim_words_list = list(sim_words[i])
        sim_words_list.append(words)
        sim_words_tuple = tuple(sim_words_list)
        list_of_words.append(sim_words_tuple)
        
    return list_of_words

input_word = 'dog,cat,mouse'
user_input = [x.strip() for x in input_word.split(',')]
result_word = []
    
for words in user_input:
    
        sim_words = model_wikipedia50.most_similar(words, topn = 0)
        sim_words = append_list(sim_words, words)
            
        result_word.extend(sim_words)
    
similar_word = [word[0] for word in result_word]
similarity = [word[1] for word in result_word] 
similar_word.extend(user_input)
labels = [word[2] for word in result_word]
label_dict = dict([(y,x+1) for x,y in enumerate(set(labels))])
color_map = [label_dict[x] for x in labels]

In [6]:
import plotly.graph_objs as go
from sklearn.decomposition import PCA

def display_pca_scatterplot_3D(model=model_wikipedia50, user_input=None, words=None, label=None, color_map=None, topn=5, sample=10):

    if words == None:
        if sample > 0:
            words = np.random.choice(list(model.vocab.keys()), sample)
        else:
            words = [ word for word in model.vocab ]
    
    word_vectors = np.array([model[w] for w in words])
    
    three_dim = PCA(random_state=0).fit_transform(word_vectors)[:,:3]
    # For 2D, change the three_dim variable into something like two_dim like the following:
    # two_dim = PCA(random_state=0).fit_transform(word_vectors)[:,:2]

    data = []
    count = 0
    
    for i in range (len(user_input)):
        trace = go.Scatter3d(
        x = three_dim[count:count+topn,0], 
        y = three_dim[count:count+topn,1],  
        z = three_dim[count:count+topn,2],
        text = words[count:count+topn],
        name = user_input[i],
        textposition = "top center",
        textfont_size = 20,
        mode = 'markers+text',
        marker = {
            'size': 10,
            'opacity': 0.8,
            'color': 2
        }

                )
                
        # For 2D, instead of using go.Scatter3d, we need to use go.Scatter and delete the z variable. Also, instead of using
        # variable three_dim, use the variable that we have declared earlier (e.g two_dim)

        data.append(trace)
        count = count+topn

    trace_input = go.Scatter3d(
                    x = three_dim[count:,0], 
                    y = three_dim[count:,1],  
                    z = three_dim[count:,2],
                    text = words[count:],
                    name = 'input words',
                    textposition = "top center",
                    textfont_size = 20,
                    mode = 'markers+text',
                    marker = {
                        'size': 10,
                        'opacity': 1,
                        'color': 'black'
                    }
                    )

    # For 2D, instead of using go.Scatter3d, we need to use go.Scatter and delete the z variable.  Also, instead of using
    # variable three_dim, use the variable that we have declared earlier (e.g two_dim)
            
    data.append(trace_input)
    
# Configure the layout

    layout = go.Layout(
        margin = {'l': 0, 'r': 0, 'b': 0, 't': 0},
        showlegend=True,
        legend=dict(     
        x=1,
        y=0.5,
        font=dict(
            family="Courier New",
            size=25,
            color="black"
        )),
        font = dict(
            family = " Courier New ",
            size = 15),
        autosize = False,
        width = 1000,
        height = 1000
        )


    plot_figure = go.Figure(data = data, layout = layout)
    plot_figure.show()
    
display_pca_scatterplot_3D(model_wikipedia50, user_input, similar_word, labels, color_map)

In [7]:
model_wikipedia50.most_similar('dog')

[('cat', 0.9218004941940308),
 ('dogs', 0.8513158559799194),
 ('horse', 0.7907583117485046),
 ('puppy', 0.7754921317100525),
 ('pet', 0.7724708318710327),
 ('rabbit', 0.7720813751220703),
 ('pig', 0.7490060925483704),
 ('snake', 0.7399188280105591),
 ('baby', 0.7395570278167725),
 ('bite', 0.7387937307357788)]

In [16]:
word_list = [model_wikipedia50.most_similar('dog')[i][0] for i in range(10)]
word_list

['cat',
 'dogs',
 'horse',
 'puppy',
 'pet',
 'rabbit',
 'pig',
 'snake',
 'baby',
 'bite']

In [23]:
words = str(word_list).replace("'","")
words = words.strip("[]")
words

'cat, dogs, horse, puppy, pet, rabbit, pig, snake, baby, bite'

In [24]:
vectors = model_wikipedia50.vectors

words = []
for i in range(400000):
    words.append(model_wikipedia50.index_to_key[i])

In [36]:
import csv 

#  write files for all words and vectors

# with open('words.tsv', 'w', newline='') as f_output:
#     tsv_output = csv.writer(f_output, delimiter='\t')
#     tsv_output.writerow(words)

# with open('vectors.tsv', 'w', newline='') as f_output:
#     tsv_output = csv.writer(f_output, delimiter='\t')
#     tsv_output.writerow(vectors)

In [38]:
len(vectors)


400000

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

%load_ext tensorboard

In [3]:
import os
import tensorflow as tf
from tensorboard.plugins import projector

In [20]:
log_dir='/Users/kate/Documents/GitHub/visual-word-embeddings/tensorboard/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = 'embedding:0'
embedding.metadata_path = 'words.tsv'

projector.visualize_embeddings(log_dir, config)

In [21]:
%tensorboard --logdir /Users/kate/Documents/GitHub/visual-word-embeddings/tensorboard/

Reusing TensorBoard on port 6006 (pid 78923), started 0:34:48 ago. (Use '!kill 78923' to kill it.)